In [ ]:
import sys
default_encoding = 'utf-8'
if sys.getdefaultencoding() != default_encoding:
    reload(sys)
    sys.setdefaultencoding(default_encoding)


In [1]:
import tushare as ts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.parser import parse
from datetime import datetime
from pandas.tseries.offsets import Hour,Day

In [3]:
col_name=['date','open','high','low','close','vol','ma5','ma10','ma20','ma60']
cyb_hist_index=pd.read_csv('c:\\pythonCoding\\data\\index\\399006.csv',header=None,skiprows=1,names=col_name,index_col='date')
#处理index的空格，使用dateutil库处理

cyb_hist_index.index=pd.to_datetime(cyb_hist_index.index)#将字符串格式的日期转换为时间序列常见的日期格式
#去掉空格数据
cyb_hist_index=cyb_hist_index.replace(cyb_hist_index.ix[cyb_hist_index.index.min(),'ma5'],np.nan)
cyb_hist_index=cyb_hist_index.dropna()

cyb_hist_index=cyb_hist_index.applymap(lambda x: float(x))
cyb_hist_index['ma5-ma20']=(cyb_hist_index['ma5']-cyb_hist_index['ma20'])/cyb_hist_index['ma20']*100

cyb_MA5_MA20=cyb_hist_index.ix["2010-08-26":"2016-03-21",'ma5-ma20']

#从tushare获取创业板指数的更新数据
startDay=cyb_hist_index.index.max()+Day()  #历史数据的最后一天+1，为tushare获取数据的开始日期
startDay=str(startDay)
Get_Cyb_Index=ts.get_hist_data('399006',start=startDay)#获取创业板指数历史信息，数据从2016-3-22开始，csv表历史数据结束于2016-3-21
ts_cyb_index=Get_Cyb_Index.ix[:,['open','high','low','close','volume','ma5','ma10','ma20']]
ts_cyb_index=ts_cyb_index.sort_index()
ts_cyb_index.index=pd.to_datetime(ts_cyb_index.index)
ts_cyb_index['ma5-ma20']=(ts_cyb_index['ma5']-ts_cyb_index['ma20'])/ts_cyb_index['ma20']*100
ts_cyb_index.columns=['open','high','low','close','vol','ma5','ma10','ma20','ma5-ma20']

cybIndex=pd.concat([cyb_hist_index,ts_cyb_index])

now=datetime.now()
date=str(now.year)+str(now.month)+str(now.day)
csvfilepath="c:\\pythoncoding\\data\\index\\cyb_index_"+date+".csv"
cybIndex.to_csv(csvfilepath)

#集合历史数据与实时数据的创业板指数MA5-MA20关系数据，此数据为下面分析的基础
cybIndex_MA5_MA20=cybIndex.ix[cyb_hist_index.index.min():ts_cyb_index.index.max(),'ma5-ma20']


#cybIndex_MA5_MA20.plot()  #绘制ma5-ma20 曲线
#plt.show()
#----------------------------------------只画创业板指数M20-M5图，到这里就可以--------------------------------

#处理创业板ETF --159915
cyb_ETF=pd.read_csv('c:\\pythonCoding\\data\\index\\159915.csv',header=None,skiprows=1,names=['date','open','high','low','close','vol','ma5','ma10','ma20','ma60'],index_col='date')
cyb_ETF.index=pd.to_datetime(cyb_ETF.index)
startDay=cyb_ETF.index.max()  #历史数据的最后一天，为tushare获取数据的开始日期
startDay=str(startDay)
get_cyb_ETF=ts.get_hist_data("159915",start=startDay)
ts_cyb_ETF=get_cyb_ETF.ix[:,['open','high','low','close','volume','ma5','ma10','ma20']]
ts_cyb_ETF=ts_cyb_ETF.sort_index()
ts_cyb_ETF.index=pd.to_datetime(ts_cyb_ETF.index)
ts_cyb_ETF['ma5-ma20']=(ts_cyb_ETF['ma5']-ts_cyb_ETF['ma20'])/ts_cyb_ETF['ma20']*100
cybETF=pd.concat([cyb_ETF,ts_cyb_ETF])

now=datetime.now()
date=str(now.year)+str(now.month)+str(now.day)
csvfilepath="c:\\pythoncoding\\data\\index\\cyb_ETF_"+date+".csv"
cybETF.to_csv(csvfilepath)


#假设初始资金量为total_initial_capital
total_initial_capital=10000



In [5]:
buy_date=cyb_MA5_MA20[cyb_MA5_MA20<=-5].index
sell_date=cyb_MA5_MA20[cyb_MA5_MA20>=5].index
buy_point=[]
sell_point=[]
for i,j in buy_date,sell_date:
    buy_point.append(buy_date[i])
    sell_point.append(sell_date[j])
    buy_date=buy_date[sell_date[j]:]

    
for i in sell_date:
    for j in buy_date:
        buy_point.append(buy_date[j])
        
    
    
    


,open,high,low,close,vol,ma5,ma10,ma20,ma60
2016-03-17,1.880,1.991,1.879,1.991,1120405888,1.904,1.875,1.921,2.107
2016-03-18,2.015,2.095,1.992,2.076,1022792064,1.952,1.902,1.92,2.097
2016-03-21,2.100,2.120,2.080,2.116,643854336,1.99,1.928,1.918,2.088
2016-03-22,2.090,2.135,2.082,2.099,673078336,2.032,1.949,1.917,2.078
2016-03-23,2.100,2.152,2.091,2.148,724549545,2.086,1.979,1.92,2.07


In [ ]:
'''
sh50index=pd.read_csv('c:\\pythonCoding\\data\\index\\000016.csv',header=None,skiprows=1,names=['date','open','high','low','close','vol','ma5','ma10','ma20','ma60'],index_col='date')
from dateutil.parser import parse
sh50index.index=pd.to_datetime(sh50index.index)#将字符串格式的日期转换为时间序列常见的日期格式
sh50index=sh50index.applymap(lambda x: float(x))
sh50index['ma5-ma20']=(sh50index['ma5']-sh50index['ma20'])/sh50index['ma20']*100
sh50index=sh50index.ix["2004-04-06":"2016-03-22",'ma5-ma20']
sh50index.plot()
plt.show()
'''

In [ ]:
for i in cybETF.index:
    if(cybIndex[i,"ma5-ma20"]<=-5):
        amount=int(total_initial_capital/cyb_ETF[i,"close"]/100)*100
        
        

















#把指数的时间段与对应ETF的时间段统一化
ETF2CYBindex_period=cybIndex_MA5_MA20[cyb_ETF.index.min():cyb_ETF.index.max()]

#买点-卖点策略判断
buy_point_value=-5 #买点值，该值可以调整优化
sell_point_value=5 #卖点值，同样可以调整优化
buy_point=ETF2CYBindex_period[cybIndex_MA5_MA20<=buy_point_value].index #5日均线-20日均线小于-5的所有日期
sell_point=cybIndex_MA5_MA20[cybIndex_MA5_MA20>=sell_point_value].index #5日均线-20日均线大于+5的所有日期




    



